In [2]:
import pandas as pd

In [58]:
races = pd.read_csv("data/races.csv")
results = pd.read_csv("data/results.csv")
driver_standings = pd.read_csv("data/driver_standings.csv")
constructor_standings = pd.read_csv("data/constructor_standings.csv")

# races = races[['raceId', 'year', 'round', 'circuitId']]
# results = results[['raceId', 'driverId', 'constructorId', 'grid', 'position']]
# driver_standings = driver_standings[['raceId', 'driverId', 'points', 'wins']]
# constructor_standings = constructor_standings[['raceId', 'constructorId', 'points', 'wins']]


In [59]:
results = results.merge(races, on="raceId", how="inner")
# Merge driver_standings on both raceId and driverId:
results = results.merge(
    driver_standings,
    on=["raceId", "driverId"],
    how="inner",
    suffixes=('', '_driver')
)
# Merge constructor_standings on both raceId and constructorId:
results = results.merge(
    constructor_standings,
    on=["raceId", "constructorId"],
    how="inner",
    suffixes=('', '_constructor')
)

In [60]:
# results = results.query("year == 2024")

results = results[['raceId', 'year', 'driverId', 'constructorId', 'round', 'grid', 'position', 'circuitId', 'points_driver', 'points_constructor', 'wins', 'wins_constructor']]

results.to_csv("cleaned/results.csv", index=False)
